In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

# ========================
# Activation functions
# ========================
def sigmoid(x):
    """Sigmoid activation: maps any real value into (0,1)"""
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    """Derivative of sigmoid given output of sigmoid"""
    return x * (1.0 - x)

# ========================
# Neural Network class
# ========================
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, lr=0.1):
        # Initialize layer sizes
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = lr  # learning rate

        # Weight matrices with small random values (0.0-1.0 range scaled)
        # Weights: input->hidden and hidden->output
        self.w1 = np.random.uniform(0.0, 1.0, (self.input_size, self.hidden_size))
        self.w2 = np.random.uniform(0.0, 1.0, (self.hidden_size, self.output_size))

        # Bias vectors initialized similarly
        self.b1 = np.random.uniform(0.0, 1.0, (1, self.hidden_size))
        self.b2 = np.random.uniform(0.0, 1.0, (1, self.output_size))

    def forward(self, X):
        # Pass inputs through the network
        self.z1 = np.dot(X, self.w1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output):
        # Calculate error at output
        output_error = y - output
        output_delta = output_error * sigmoid_derivative(output)

        # Calculate error at hidden layer
        hidden_error = output_delta.dot(self.w2.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.a1)

        # Update weights and biases
        self.w2 += self.a1.T.dot(output_delta) * self.lr
        self.b2 += np.sum(output_delta, axis=0, keepdims=True) * self.lr
        self.w1 += X.T.dot(hidden_delta) * self.lr
        self.b1 += np.sum(hidden_delta, axis=0, keepdims=True) * self.lr

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
            if (epoch + 1) % 100 == 0:
                loss = np.mean((y - output) ** 2)
                print(f"Epoch {epoch+1}/{epochs} - Loss: {loss:.4f}")

    def predict(self, X):
        # Return class with highest output activation
        probs = self.forward(X)
        return np.argmax(probs, axis=1), probs

# ========================
# Data preparation
# ========================
# Load Iris dataset
iris = datasets.load_iris()
X = iris.data  # shape (150,4)
y = iris.target  # class labels 0,1,2

# One-hot encode targets
y_encoded = np.zeros((y.size, y.max()+1))
y_encoded[np.arange(y.size), y] = 1

# Scale features to [0.05, 0.95]
def scale_features(x):
    min_val = x.min(axis=0)
    max_val = x.max(axis=0)
    # normalize to [0,1]
    x_norm = (x - min_val) / (max_val - min_val)
    # scale to [0.05,0.95]
    return x_norm * 0.9 + 0.05

X_scaled = scale_features(X)


import numpy as np
from sklearn.model_selection import train_test_split

# --- your synthetic inputs and targets ---
x0 = np.array([0.9, 0.7, 0.3, 0.6])
x1 = np.array([0.6, 0.7, 0.6, 0.7])
X = np.vstack([x0 if i % 2 == 0 else x1 for i in range(150)])  # shape (150,4)

y0 = np.array([2.95, 2.0, 2.0])   # Class 0 target
y1 = np.array([2.0, 2.0, 2.95])   # Class 2 target
Y = np.vstack([y0 if i % 2 == 0 else y1 for i in range(150)])  # shape (150,3)

# --- split into 80/20 train/test ---
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=[0 if i%2==0 else 1 for i in range(150)]
)

# --- build & train your network ---
nn = NeuralNetwork(input_size=4, hidden_size=10, output_size=3, lr=0.1)
nn.train(X_train, y_train, epochs=10000)

# --- evaluate on the held‐out 20% ---
y_pred, y_probs = nn.predict(X_test)

# if you want an “accuracy”‐style measure, you need class indices:
y_true_idx = np.argmax(y_test, axis=1)
y_pred_idx = y_pred
accuracy = np.mean(y_pred_idx == y_true_idx)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 100/10000 - Loss: 1.9343
Epoch 200/10000 - Loss: 1.9343
Epoch 300/10000 - Loss: 1.9342
Epoch 400/10000 - Loss: 1.9342
Epoch 500/10000 - Loss: 1.9342
Epoch 600/10000 - Loss: 1.9342
Epoch 700/10000 - Loss: 1.9342
Epoch 800/10000 - Loss: 1.9342
Epoch 900/10000 - Loss: 1.9342
Epoch 1000/10000 - Loss: 1.9342
Epoch 1100/10000 - Loss: 1.9342
Epoch 1200/10000 - Loss: 1.9342
Epoch 1300/10000 - Loss: 1.9342
Epoch 1400/10000 - Loss: 1.9342
Epoch 1500/10000 - Loss: 1.9342
Epoch 1600/10000 - Loss: 1.9342
Epoch 1700/10000 - Loss: 1.9342
Epoch 1800/10000 - Loss: 1.9342
Epoch 1900/10000 - Loss: 1.9342
Epoch 2000/10000 - Loss: 1.9342
Epoch 2100/10000 - Loss: 1.9342
Epoch 2200/10000 - Loss: 1.9342
Epoch 2300/10000 - Loss: 1.9342
Epoch 2400/10000 - Loss: 1.9342
Epoch 2500/10000 - Loss: 1.9342
Epoch 2600/10000 - Loss: 1.9342
Epoch 2700/10000 - Loss: 1.9342
Epoch 2800/10000 - Loss: 1.9342
Epoch 2900/10000 - Loss: 1.9342
Epoch 3000/10000 - Loss: 1.9342
Epoch 3100/10000 - Loss: 1.9342
Epoch 3200/10000 